<a href="https://colab.research.google.com/github/SebastianLarssonDTU/02456-Reinforcement-Learning-Project/blob/before_framestacking/Combining_from_multiple_csv.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# INIT : Procgen, Drive, Git

In [1]:
!pip install procgen

In [2]:
#Clone git
!git clone -b restructure_code https://github.com/SebastianLarssonDTU/02456-Reinforcement-Learning-Project.git "my_project"

fatal: destination path 'my_project' already exists and is not an empty directory.


In [3]:
#update git
%cd /content/my_project
! git pull

/content/my_project
Already up to date.


In [4]:
import datatools as tools
from datatools import DATA_PATH, MODEL_PATH
#Mount drive
tools.mount_drive()

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# Import

In [5]:
#Import all custom files
import baseline
import datatools as tools
import hyperparameters as h
import model
import my_util
import policy
import ppo
import utils

#other imports
import torch

#import specific methods
from baseline import set_hyperparameters
from ppo import PPO
from experiments import run_experiment, print_list_of_experiments


In [6]:
print_list_of_experiments()

***** Experiment 0 *****
Description:    Baseline inspired by PPO article
***** Experiment 1 *****
Description:    Baseline inspired by Procgen article
***** Experiment 2 *****
Description:    Modified PPO baseline with value clipping enabled
***** Experiment 3 *****
Description:    Modified PPO baseline with value clipping enabled and learning rate given by par variable
***** Experiment 4 *****
Description:    Modified PPO baseline with value clipping enabled and reward penalty on death (1 as default)
***** Experiment 5 *****
Description:    Baseline inspired by IMPALA paper (No value clipping)
***** Experiment 6 *****
Description:    Inspired by IMPALA paper (With value clipping)
***** Experiment 7 *****
Description:    Inspired by both IMPALA and Procgen papers (With value clipping)
***** Experiment 8 *****
Description:    Inspired by both IMPALA and Procgen papers (With value clipping) and added death penalty


# Create Index File



In [8]:
INDEX = tools.create_index_table_from_txt_files()
INDEX

,total_steps,num_envs,num_levels,num_steps,num_epochs,batch_size,eps,grad_eps,value_coef,entropy_coef,lr,gamma,lmbda,version,time_limit,value_clipping,death_penalty,file_name,eval,eval_cycle,Time spent (in seconds),Steps taken,penalty,save_interval,step_start,nstack,Time spent,Last Mean Reward
Value,8000000.0,32,10,256,3,512,0.1,0.5,1,0.01,0.00025,0.99,0.95,Experiment0,43200,False,False,Experiment0_Run_26Nov_15h21m41s,True,16,18858.62,8003584,NaN,NaN,NaN,NaN,5:14:18,20.34375
Value,8000000.0,32,10,256,3,512,0.2,0.5,0.5,0.01,0.0005,0.999,0.95,Experiment1,43200,False,False,Experiment1_Run_26Nov_15h22m38s,True,16,19818.86,8003584,NaN,NaN,NaN,NaN,5:30:18,19.53125
Value,8000000.0,32,10,256,3,512,0.2,0.5,0.5,0.01,0.0005,0.999,0.95,Experiment2,43200,True,False,Experiment2_Run_26Nov_22h02m36s,True,16,20771.90,8003584,NaN,NaN,NaN,NaN,5:46:11,23.96875
Value,8000000.0,32,10,256,3,512,0.1,0.5,1,0.01,0.00025,0.99,0.95,Experiment2,43200,True,False,Experiment2_Run_27Nov_11h05m48s,True,16,20480.86,8003584,NaN,NaN,NaN,NaN,5:41:20,28.15625
Value,8000000.0,32,10,256,3,512,0.1,0.5,1,0.01,0.00025,0.99,0.95,Experiment4,43200,True,True,Experiment4_Run_27Nov_11h07m20s,True,16,20819.16,8003584,NaN,NaN,NaN,NaN,5:46:59,25.8125
Value,8000000.0,32,10,256,3,512,0.1,0.5,1,0.01,0.0025,0.99,0.95,Experiment3,43200,True,False,Experiment3_Run_27Nov_18h03m39s,True,16,NaN,4136960,NaN,NaN,NaN,NaN,NaN,23.0
Value,8000000.0,32,10,256,3,512,0.1,0.5,1,0.01,0.00025,0.99,0.95,Experiment4,43200,True,True,Experiment4_Run_27Nov_18h11m00s,True,16,NaN,7077888,3,NaN,NaN,NaN,NaN,24.5
Value,8000000.0,32,10,256,3,32,0.2,0.5,0.5,0.01,0.0006,0.99,0.95,Experiment5,43200,False,False,Experiment5_Run_27Nov_16h37m42s,True,16,32130.90,8003584,NaN,NaN,NaN,NaN,8:55:30,16.28125
Value,8000000.0,32,10,256,3,32,0.2,0.5,0.5,0.01,0.0006,0.99,0.95,Experiment6,43200,True,False,Experiment6_Run_28Nov_12h09m43s,True,16,34241.34,8003584,1,NaN,NaN,NaN,9:30:41,24.15625
Value,8000000.0,32,10,256,3,512,0.1,0.5,1,0.01,2.5e-05,0.99,0.95,Experiment3,43200,True,False,Experiment3_Run_29Nov_16h30m27s,True,16,NaN,6979584,1,NaN,NaN,NaN,NaN,22.03125


In [9]:
import glob

file_names = [x.strip() for x in INDEX['file_name']]
n = len(file_names)
# for i in range(n):
#   file_names.append(INDEX['file_name'][-1-i].strip())

In [10]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

def plot_training_graphs(file_names, legend = None):
  df = pd.DataFrame(columns=file_names)
  df['Step'] = pd.read_csv(DATA_PATH + INDEX["file_name"][0].strip()+".csv")["Step"]  #This only works as long as they all have the same steps!
  for file_name in file_names:
    df[file_name] = pd.read_csv(DATA_PATH + file_name+".csv")[" Mean reward"]
  
  df = df.set_index('Step')

  plt.figure(figsize=(16, 10))
  sns.lineplot(data = df)
  if legend is not None:
    plt.legend(labels=[legend[x] for x in file_names])
  plt.title("Training")

def plot_eval_graphs(file_names, legend = None):
  df = pd.DataFrame(columns=file_names)
  for file_name in file_names:
    temp = pd.read_csv(DATA_PATH + file_name+"_EVAL.csv")
    df[file_name] = temp["avg"]
  plt.figure(figsize=(16, 10))
  sns.lineplot(data = df)
  if legend is not None:
    plt.legend(labels=[legend[x] for x in file_names])
  plt.title("Eval")
  # return df

In [11]:
for i in range(len(file_names)):
  print("[{:2d}]: {}".format(i, file_names[i]))

[ 0]: Experiment0_Run_26Nov_15h21m41s
[ 1]: Experiment1_Run_26Nov_15h22m38s
[ 2]: Experiment2_Run_26Nov_22h02m36s
[ 3]: Experiment2_Run_27Nov_11h05m48s
[ 4]: Experiment4_Run_27Nov_11h07m20s
[ 5]: Experiment3_Run_27Nov_18h03m39s
[ 6]: Experiment4_Run_27Nov_18h11m00s
[ 7]: Experiment5_Run_27Nov_16h37m42s
[ 8]: Experiment6_Run_28Nov_12h09m43s
[ 9]: Experiment3_Run_29Nov_16h30m27s
[10]: Experiment7_Run_29Nov_16h32m47s
[11]: Experiment7_Run_30Nov_14h45m32s
[12]: Experiment8_Run_30Nov_22h21m06s
[13]: Experiment7_50levels_Run_01Dec_07h28m45s
[14]: Experiment2_50levels_Run_01Dec_10h07m41s
[15]: Experiment2_100levels_Run_02Dec_09h27m30s
[16]: Experiment7_100levels_Run_02Dec_09h29m48s
[17]: Experiment8_50levels_Run_02Dec_17h18m34s
[18]: Experiment8_10levels_Run_02Dec_09h48m26s
[19]: Experiment2_100levels_Run_03Dec_10h48m24s
[20]: Experiment8_50levels_Run_03Dec_14h53m09s
[21]: Experiment8_10levels_Run_03Dec_11h04m34s
[22]: Experiment8_50levels_Run_04Dec_06h17m22s
[23]: Experiment4_50levels_Run_04

# Combine files from multiple runs (PPO)

In [12]:
root_name = "Experiment2_50levels_Run_04Dec_18h10m32s"

## Print related files

In [15]:
all_related_files= []
for name in file_names:
  if root_name in name:
    all_related_files.append(name)

for name in all_related_files:
  print(name)

Experiment2_50levels_Run_04Dec_18h10m32s
Experiment2_50levels_Run_04Dec_18h10m32s_loaded_05Dec_09h43m36s
Experiment2_50levels_Run_04Dec_18h10m32s_loaded_06Dec_12h08m48s


In [25]:
import glob
all_models=glob.glob(MODEL_PATH +"*.pt")
for model_path in all_models:
  if root_name in model_path:
    print(model_path)

/content/drive/My Drive/02456-Deep-Learning-Project/Models/Experiment2_50levels_Run_04Dec_18h10m32s_507904steps.pt
/content/drive/My Drive/02456-Deep-Learning-Project/Models/Experiment2_50levels_Run_04Dec_18h10m32s_1007616steps.pt
/content/drive/My Drive/02456-Deep-Learning-Project/Models/Experiment2_50levels_Run_04Dec_18h10m32s_1507328steps.pt
/content/drive/My Drive/02456-Deep-Learning-Project/Models/Experiment2_50levels_Run_04Dec_18h10m32s_2007040steps.pt
/content/drive/My Drive/02456-Deep-Learning-Project/Models/Experiment2_50levels_Run_04Dec_18h10m32s_2506752steps.pt
/content/drive/My Drive/02456-Deep-Learning-Project/Models/Experiment2_50levels_Run_04Dec_18h10m32s_3006464steps.pt
/content/drive/My Drive/02456-Deep-Learning-Project/Models/Experiment2_50levels_Run_04Dec_18h10m32s_3506176steps.pt
/content/drive/My Drive/02456-Deep-Learning-Project/Models/Experiment2_50levels_Run_04Dec_18h10m32s_4005888steps.pt
/content/drive/My Drive/02456-Deep-Learning-Project/Models/Experiment2_50

**CONCLUSION:**

First load at 6004736steps

Second load at 13516800steps which is actually 13516800-6004736steps = 

In [29]:
13516800-6004736

7512064

In [26]:
for model_path in all_models:
  if root_name in model_path and "loaded" in model_path:
    print(model_path)

/content/drive/My Drive/02456-Deep-Learning-Project/Models/Experiment2_50levels_Run_04Dec_18h10m32s_loaded_05Dec_09h43m36s_12517376steps.pt
/content/drive/My Drive/02456-Deep-Learning-Project/Models/Experiment2_50levels_Run_04Dec_18h10m32s_loaded_05Dec_09h43m36s_13017088steps.pt
/content/drive/My Drive/02456-Deep-Learning-Project/Models/Experiment2_50levels_Run_04Dec_18h10m32s_loaded_05Dec_09h43m36s_13516800steps.pt
/content/drive/My Drive/02456-Deep-Learning-Project/Models/Experiment2_50levels_Run_04Dec_18h10m32s_loaded_06Dec_12h08m48s_27541504steps.pt
/content/drive/My Drive/02456-Deep-Learning-Project/Models/Experiment2_50levels_Run_04Dec_18h10m32s_loaded_06Dec_12h08m48s_28041216steps.pt
/content/drive/My Drive/02456-Deep-Learning-Project/Models/Experiment2_50levels_Run_04Dec_18h10m32s_loaded_06Dec_12h08m48s_28540928steps.pt
/content/drive/My Drive/02456-Deep-Learning-Project/Models/Experiment2_50levels_Run_04Dec_18h10m32s_loaded_06Dec_12h08m48s_29040640steps.pt
/content/drive/My Dr

## Create dfs

Notice overlap since the model is not necessarily saved in the end, model can have timed out or been manually stopped

In [35]:
dfs = []
for name in all_related_files:
  dfs.append(pd.read_csv(DATA_PATH + name + ".csv"))

#Correct for error in name
dfs[2]["Step"] =dfs[2]["Step"]-6004736
for df in dfs:
  print("******************")
  print(df.head())
  print(df.tail())

******************
    Step   Mean reward
0   8192       4.59375
1  16384       4.93750
2  24576       4.96875
3  32768       4.34375
4  40960       5.78125
        Step   Mean reward
763  6258688      20.96875
764  6266880      23.40625
765  6275072      21.25000
766  6283264      21.81250
767  6291456      20.75000
******************
      Step   Mean reward
0  6012928      19.87500
1  6021120      22.78125
2  6029312      21.78125
3  6037504      21.18750
4  6045696      21.21875
        Step   Mean reward
190  7569408      21.56250
191  7577600      21.37500
192  7585792      21.09375
193  7593984      20.87500
194  7602176      20.62500
******************
      Step   Mean reward
0  7520256      20.71875
1  7528448      21.50000
2  7536640      23.18750
3  7544832      22.65625
4  7553024      22.87500
         Step   Mean reward
313  10084352      24.00000
314  10092544      23.46875
315  10100736      22.53125
316  10108928      22.87500
317  10117120      24.15625


# Combine files from multiple runs (Impala)

In [36]:
root_name = "Experiment7_50levels_Run_04Dec_18h12m45s"

## Print related files

In [37]:
all_related_files= []
for name in file_names:
  if root_name in name:
    all_related_files.append(name)

for name in all_related_files:
  print(name)

Experiment7_50levels_Run_04Dec_18h12m45s
Experiment7_50levels_Run_04Dec_18h12m45s_loaded_05Dec_10h49m07s
Experiment7_50levels_Run_04Dec_18h12m45s_loaded_06Dec_12h10m21s


In [38]:
import glob
all_models=glob.glob(MODEL_PATH +"*.pt")
for model_path in all_models:
  if root_name in model_path:
    print(model_path)

/content/drive/My Drive/02456-Deep-Learning-Project/Models/Experiment7_50levels_Run_04Dec_18h12m45s_507904steps.pt
/content/drive/My Drive/02456-Deep-Learning-Project/Models/Experiment7_50levels_Run_04Dec_18h12m45s_1007616steps.pt
/content/drive/My Drive/02456-Deep-Learning-Project/Models/Experiment7_50levels_Run_04Dec_18h12m45s_1507328steps.pt
/content/drive/My Drive/02456-Deep-Learning-Project/Models/Experiment7_50levels_Run_04Dec_18h12m45s_2007040steps.pt
/content/drive/My Drive/02456-Deep-Learning-Project/Models/Experiment7_50levels_Run_04Dec_18h12m45s_2506752steps.pt
/content/drive/My Drive/02456-Deep-Learning-Project/Models/Experiment7_50levels_Run_04Dec_18h12m45s_3006464steps.pt
/content/drive/My Drive/02456-Deep-Learning-Project/Models/Experiment7_50levels_Run_04Dec_18h12m45s_3506176steps.pt
/content/drive/My Drive/02456-Deep-Learning-Project/Models/Experiment7_50levels_Run_04Dec_18h12m45s_4005888steps.pt
/content/drive/My Drive/02456-Deep-Learning-Project/Models/Experiment7_50

**CONCLUSION:**

First load at 4505600steps

Second load at 13516800steps which is actually 13516800-4505600steps = 

In [39]:
13516800-4505600

9011200

In [40]:
for model_path in all_models:
  if root_name in model_path and "loaded" in model_path:
    print(model_path)

/content/drive/My Drive/02456-Deep-Learning-Project/Models/Experiment7_50levels_Run_04Dec_18h12m45s_loaded_05Dec_10h49m07s_9519104steps.pt
/content/drive/My Drive/02456-Deep-Learning-Project/Models/Experiment7_50levels_Run_04Dec_18h12m45s_loaded_05Dec_10h49m07s_10018816steps.pt
/content/drive/My Drive/02456-Deep-Learning-Project/Models/Experiment7_50levels_Run_04Dec_18h12m45s_loaded_05Dec_10h49m07s_10518528steps.pt
/content/drive/My Drive/02456-Deep-Learning-Project/Models/Experiment7_50levels_Run_04Dec_18h12m45s_loaded_05Dec_10h49m07s_11018240steps.pt
/content/drive/My Drive/02456-Deep-Learning-Project/Models/Experiment7_50levels_Run_04Dec_18h12m45s_loaded_05Dec_10h49m07s_11517952steps.pt
/content/drive/My Drive/02456-Deep-Learning-Project/Models/Experiment7_50levels_Run_04Dec_18h12m45s_loaded_05Dec_10h49m07s_12017664steps.pt
/content/drive/My Drive/02456-Deep-Learning-Project/Models/Experiment7_50levels_Run_04Dec_18h12m45s_loaded_05Dec_10h49m07s_12517376steps.pt
/content/drive/My Dri

## Create dfs

Notice overlap since the model is not necessarily saved in the end, model can have timed out or been manually stopped

In [41]:
dfs = []
for name in all_related_files:
  dfs.append(pd.read_csv(DATA_PATH + name + ".csv"))

#Correct for error in name
dfs[2]["Step"] =dfs[2]["Step"]-4505600
for df in dfs:
  print("******************")
  print(df.head())
  print(df.tail())

******************
    Step   Mean reward
0   8192       4.93750
1  16384       5.15625
2  24576       5.84375
3  32768       6.37500
4  40960       7.43750
        Step   Mean reward
547  4489216      24.28125
548  4497408      24.65625
549  4505600      24.25000
550  4513792      23.31250
551  4521984      24.71875
******************
      Step   Mean reward
0  4513792      22.37500
1  4521984      22.71875
2  4530176      25.03125
3  4538368      23.28125
4  4546560      26.06250
        Step   Mean reward
581  9273344      27.09375
582  9281536      25.31250
583  9289728      25.84375
584  9297920      25.90625
585  9306112      23.93750
******************
      Step   Mean reward
0  9019392      23.03125
1  9027584      24.62500
2  9035776      28.00000
3  9043968      26.28125
4  9052160      26.43750
         Step   Mean reward
249  11059200      26.12500
250  11067392      25.40625
251  11075584      26.18750
252  11083776      27.00000
253  11091968      27.21875
